In [1]:
from elasticsearch import Elasticsearch
import numpy as np
import pandas as pd
import scipy as sc
import sklearn as skl
import itertools
import ast
import datetime
import nltk
import os
import io
import re
import csv
import operator
import warnings
from nltk.corpus import stopwords
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from pandas import *
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
from elasticsearch import helpers
from elasticsearch.helpers import bulk
from datetime import datetime
warnings.filterwarnings('ignore')

In [2]:
from collections import Counter

In [3]:
import math

# Data Preprocessing

In [4]:
with open('./all_articles.txt','r',encoding='utf-8',errors='ignore') as txt:
    corpus = ast.literal_eval(txt.read())

In [5]:
#ergodic the txt
corpus = list(itertools.chain(*corpus))
corpus = list(itertools.chain(*corpus))

In [6]:
#show the ergodic result
corpus[0]

'Earlier today we had a strong South Korean PMI report.'

In [7]:
#Set stop words
stop_words = set(stopwords.words('english'))

In [8]:
#establish the Elasticsearch
es_articles = Elasticsearch()

In [9]:
#insert the article to Elasticsearch,Sentence by sentence
document = []
for i in range(len(corpus)):
    sentences = corpus[i]
    documents = {
        "_index" : "articles_all",
        "_type" : "article_type",
        "_id" : i,
        "_source" :{
            "any" : "data" + str(i),
            "timestamp" : datetime.now(),
            "body" : sentences
        }
    }
    document.append(documents)
if len(document) > 0:
    helpers.bulk(es_articles , document)


# Raise Entities

In [30]:
#Raise entities 
#Use the nltk.word_tokenize ,pos_tag and ne_chunk to raise entities
def rais_entities(sent):
    if type(sent) == str:
        nerEntities = nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent)))
    else:
        nerEntities = nltk.ne_chunk(nltk.pos_tag(sent))
    result = []
    current = []
    #Accord NerEntities to decide the nerEntities's Object
    #If nerEntities is Tree put the keywords into the current
    for i in nerEntities:
        if type(i) == Tree:
            current.append(" ".join([token for token , pos in i.leaves()]))
        else:
            #if not , put the current into named_entities
            named_entity = " ".join(current)
            #then put the named_entities into teh result
            if named_entity not in result:
                result.append(named_entity)
                current = []
    return result

In [31]:
rais_entities('which companies went bankrupt in month X of year Y?')

['']

In [32]:
rais_entities('who is the CEO of company Apple?')

['', 'CEO of', 'Apple']

In [23]:
#show questions
question1 = "which companies went bankrupt in month X of year Y?"
question2 = "what affects GDP?"
question3 = "what percentage of drop or increase is associated with this property?"
question4 = "who is the CEO of company X?"

In [24]:
#Use compile transfer str to unicode
words = re.compile(r'\w+')

In [25]:
words

re.compile(r'\w+', re.UNICODE)

In [26]:
#user the unicode transform the question
question_vec1 = Counter(words.findall(question1))
question_vec2 = Counter(words.findall(question2))
question_vec3 = Counter(words.findall(question3))
question_vec4 = Counter(words.findall(question4))

In [27]:
#Result of transformation
question_vec2

Counter({'what': 1, 'affects': 1, 'GDP': 1})

# Classify Questions

In [36]:
#Classify Questions: CEO "3", Bankrupt "0", GDP "1"
#calculate Cosine to decide question belone to which question model
def classifier_question(question):
    question = Counter(words.findall(question))
    cosine_array = np.zeros(4)
    #####################question1 and question#######################
    intersection1 = set(question_vec1.keys())& set(question.keys())
    #print (intersection1)
    numerator1 =sum([question_vec1[x]*question[x] for x in intersection1])
    #print (numerator1)
    sum1a = sum([question_vec1[x]**2 for x in question_vec1.keys()])
    #print (sum1a)
    sum1b = sum([question[x] ** 2 for x in question.keys()])
    #print (sum1b)
    denominator1 = math.sqrt(sum1a) * math.sqrt(sum1b)
    #print (denominator1)
    if not denominator1:
        cosine_array[0] = 0.0
    else:
        cosine_array[0] = float(numerator1)/denominator1
    #print (cosine_array[0])
    #############################################################
    ######################question2 and question#######################
    intersection2 = set(question_vec2.keys())& set(question.keys())
    numerator2 =sum([question_vec2[x]*question[x] for x in intersection2])
    
    sum2a = sum([question_vec2[x]**2 for x in question_vec2.keys()])
    sum2b = sum([question[x] ** 2 for x in question.keys()])
    denominator2 = math.sqrt(sum2a) * math.sqrt(sum2b)
    if not denominator2:
        cosine_array[1] = 0.0
    else:
        cosine_array[1] = float(numerator2)/denominator2
    #############################################################
    ######################question3 and question#######################    
    intersection3 = set(question_vec3.keys())& set(question.keys())
    numerator3 =sum([question_vec3[x]*question[x] for x in intersection3])
    
    sum3a = sum([question_vec3[x]**2 for x in question_vec3.keys()])
    sum3b = sum([question[x] ** 2 for x in question.keys()])
    denominator3 = math.sqrt(sum3a) * math.sqrt(sum3b)
    if not denominator3:
        cosine_array[2] = 0.0
    else:
        cosine_array[2] = float(numerator3)/denominator3
    #############################################################
    ######################question4 and question#######################      
    intersection4 = set(question_vec4.keys())& set(question.keys())
    numerator4 =sum([question_vec4[x]*question[x] for x in intersection4])
    
    sum4a = sum([question_vec4[x]**2 for x in question_vec4.keys()])
    sum4b = sum([question[x] ** 2 for x in question.keys()])
    denominator4 = math.sqrt(sum4a) * math.sqrt(sum4b)
    if not denominator4:
        cosine_array[3] = 0.0
    else:
        cosine_array[3] = float(numerator4)/denominator4
    #show the maximal number's index
    max_index = np.argmax(cosine_array)
    
    return max_index
    

In [37]:
#Test this define
questiontype = classifier_question("What percentage of drop or increase is associated with this property?")

In [38]:
#questiontype= 2 it means this question belone with question model-3
questiontype

2

# Define the function about find CEO name

In [39]:
def find_CEO(question):
    #Raise entities ，the result like 'CEO OF ***'
    nameEnitities = rais_entities(question)
    
    #Use Elasticsearch to search result
    ESsearch = es_articles.search(index='articles_all' , q = nameEnitities , size = 1000)
    
    #take out the sentence from ESsearch
    ES_result = []
    for i in np.arange(len(ESsearch['hits']['hits'])):
        ES_result.append(ESsearch['hits']['hits'][i]['_source']['body'])
        ES_result[i] = nltk.word_tokenize(ES_result[i])
        
    #word_tokenize the nameEntities 
    #find the company name from question
    tq = nltk.word_tokenize(nameEnitities[0])
    if 'of' in tq:
        tq.remove('of')
    if len(tq) == 1:
        companyname = tq[0]
    else:
        companyname = tq[1]
    if len(companyname) > 2:
        for i in range(2, len(tq)):
            companyname = companyname + ' ' + tq[i]
    
    #find sentence form ES_result what about company and CEO
    about_company = []
    for i in np.arange(len(ES_result)):
        switch = 1
        for j in np.arange(len(tq)):
            if tq[j] not in ES_result[i]:
                switch = 0
        if switch == 1:
            about_company.append(ES_result[i])
        
    #deal with about_company , use rais_ectities get keywords from about_company
    #resule_ceo is all about caompany and CEO.
    result_ceo = []
    for i in np.arange(len(about_company)):
        entities_result = rais_entities(about_company[i])
        for j in np.arange(len(entities_result)):
            if (entities_result[j] != companyname) and (len(nltk.word_tokenize(entities_result[j])))==2:
                result_ceo.append(entities_result[j])
    
    
    #If 'companyname' 'CEO' in the same sentence , teh CEO name have large probability in this sentence
    #use max(set,key) function find the CEO_name
    return max(set(result_ceo),key = result_ceo.count)
    

In [409]:
#test this function
find_CEO('What is the CEO of Facebook?')

'Mark Zuckerberg'

In [410]:
find_CEO('What is the CEO of Amazon?')

'Jeff Bezos'

In [421]:
find_CEO("Who is the CEO of Yahoo?")

'Marissa Mayer'

# Define the function to find bankruptcy company

In [294]:
from elasticsearch_dsl import Search, query

In [96]:
def find_bankruptcy(question):
    monthdict = {'January':1, 'February':2, 'March':3, 'April':4,
                'May':5, 'June':6, 'July':7, 'August':8,
                'September':9 , 'October':10 , 'November':11 , 'December':12}
    #word_tokenize the question
    tq = nltk.word_tokenize(question)
    
    #Use dict to find month
    month = list(filter(lambda x:x in tq , monthdict))[0]
    
    # Use RE to find yeas
    years = re.findall(r'\d{4}' , question)
    if years:
        year = years[0]
    else:
        return "Please provide which year"
    
    #put the moth and year into the datequrey
    datequrey = []
    datequrey.append(month)
    datequrey.append(years[0])
    print (datequrey)
    
    #Use Elasticsearch to find result about 'moth' and 'yesr'
    ESsearch = es_articles.search(index='articles_all' , q = datequrey ,size = 1000)
    
    #find sentence form Essearch what about month and year
    out_hits = ESsearch['hits']['hits']
    for_sentence = []
    for i in np.arange(len(out_hits)):
        sent = out_hits[i]['_source']['body']
        
        #word_tokenize the sent
        ts = nltk.word_tokenize(sent)
        
        #delete the stop words
        ts = [token for token in ts if not token in stop_words]
        
        #If we want to find which year and month the sentence must have 'month','year' 
        # and one of the 'key_words = ['bankruptcy','bankrupt','broke']''
        if datequrey[0] in ts and datequrey[1] in ts and ('bankruptcy' in ts or 'bankrupt' in ts or 'broke' in ts):
            for_sentence.append(sent)
            #print (for_sentence)
    candidates = []
    for i in np.arange(len(for_sentence)):
        ne = rais_entities(for_sentence[i])
        for j in np.arange(len(ne)):
            if ne[j] != '':
                candidates.append(ne[j].lower())
    #print (candidates)
    #use max(set,key) function find the company_name
    return max(set(candidates), key = candidates.count)
      
            
    

In [97]:
find_bankruptcy("Which company went bankrupt in September 2008?")

['September', '2008']


'lehman brothers'

In [98]:
find_bankruptcy("Which company went bankrupt in October 2014?")

['October', '2014']


'detroit'

In [99]:
find_bankruptcy("Which company went bankrupt in October 2008?")

['October', '2008']


'seeitmarket'

# Define the function to find what affect GDP

In [923]:
def affect_GDP(question):
    
    #Analysis question
    tq = nltk.word_tokenize(question)
    if '?' in tq:
        tq.remove('?')
    pos_question = nltk.pos_tag(tq)
    keywords = []
    for i in range(len(tq)-1,-1,-1):
        if pos_question[i][1] != 'TO' and pos_question[i][1] !='IN':
            keywords.insert(0,tq[i])
        else:
            break
   
    #definr the keywords about affect and GDP
    GDPquery = ['GDP','effect','effects','affect','affects']
    ESsearch = es_articles.search(index='articles_all' , q = GDPquery ,size=1000)
    GDPresult = ESsearch['hits']['hits']
    
    #raise sentence form ESsearch
    candgdp = []
    for i in np.arange(len(GDPresult)):
        sent = GDPresult[i]['_source']['body']
        ts = nltk.word_tokenize(sent)
        
        #delete stopwords
        ts = [token for token in ts if not token in stop_words]
        candgdp.append(ts)
    pos_taglist = []
    
    #if one sentence have 'GDP' and one of ('effect','effects','affect','affects')
    #raise them and use pos_tag deal with them
    for i in range(len(candgdp)):
        if 'GDP' in candgdp[i] and ('effect' in candgdp[i] or 'affect' in candgdp[i] or 'effects' in candgdp[i] or 'affects' in candgdp[i]):
            pt = nltk.pos_tag(candgdp[i])
            pos_taglist.append(pt)
    
    #raise 'NN' from pos_taglist
    NN_list = []
    for i in range(len(pos_taglist)):
        for j in range(len(pos_taglist[i])):
            if 'NN'in pos_taglist[i][j] :
                
                NN_list.append(pos_taglist[i][j])
                #print (pos_taglist[i][j])
    
    #deal with NN_list , delete symbol
    candresult = []
    for i in range(len(NN_list)):
        candresult.append(str(NN_list[i]).replace('NN' , '').replace('\'','').replace(',','').replace('(','').replace(')',''))
    aaa = []
    for i in range(len(candresult)):
        
        #delete ('effect','effects','affect','affects')
        if 'effect' not in candresult[i] and 'affect' not in candresult[i] and 'effects' not in candresult[i] and 'affects' not in candresult[i] and 'growth' not in candresult[i] and '%' not in candresult[i]:
            aaa.append(candresult[i])
    
    
    if 'most' in tq:
        return (max(set(aaa),key = candresult.count))
    else:
        return("Policy,hurricane,Consumption, consumer spending, government spending, investment, imports, exports, foreign trade,tax")
    
    
    
    

In [647]:
#Test this function
affect_GDP("what the  affects GDP?")

'Policy,Consumption, consumer spending, government spending, investment, imports, exports, foreign trade,tax'

In [648]:
affect_GDP("what the most affects GDP?")

'policy '

# Define the function to deal with“what percentage of drop or increase is associated with this property?”

In [921]:
def find_percent(question):
    #List out factor about GDP
    factor = 'Policy,Consumption, consumer spending, government spending, investment, imports, exports, foreign trade,tax'
    
    #word_tokenize the factor
    wt_factor = nltk.word_tokenize(factor.lower())
    
    #word_tokenize the question
    wt_question = nltk.word_tokenize(question)
    
    #Keywords is the factor what question asked
    keywords = []
    for i in range(len(wt_factor)):
        for j in range(len(wt_question)):
            if wt_factor[i] == wt_question[j]:
                keywords.append(wt_factor[i])
    
    #Clean the keywords
    if len(keywords) == 3:
        keywords = keywords[0]+' '+keywords[1]
        
    
    if keywords != []:
        
        #Clean keywords again
        qkeyword = str(keywords).replace('[','').replace(']','').replace('\'','').replace(',' , '')
        
        #make 'q' to use Elasticsearch
        #keyword may have these words
        keyword = [qkeyword,'GDP','%','percent','percentage','increase','decrease','growth','effect','effects','affect','affects']
        
        #Use Elasticsearch
        ESsearch = es_articles.search(index = 'articles_all' , q = keyword ,size = 1000)
        
        #raise the sentence from ESsearch
        ES_result = []
        for i in np.arange(len(ESsearch['hits']['hits'])):
            ES_result.append(ESsearch['hits']['hits'][i]['_source']['body'])
            ES_result[i] = nltk.word_tokenize(ES_result[i])
        
        #udeg the keywords from question , len=1 or len=2
        candsent = []
        wtkey = nltk.word_tokenize(qkeyword)
        for i in range(len(ES_result)):
            if len(wtkey) == 2:
                
                if 'GDP' in ES_result[i] and wtkey[0] in ES_result[i] and wtkey[1] in ES_result[i] and ('%' in ES_result[i] or 'percent' in ES_result[i] or 'percentage' in ES_result[i]) and ('adding' in ES_result[i] or 'growth' in ES_result[i]):
                    candsent.append(ES_result[i])
            if len(wtkey) == 1:
                if 'GDP' in ES_result[i] and qkeyword in ES_result[i] and ('%' in ES_result[i] or 'percent' in ES_result[i] or 'percentage' in ES_result[i]):
                    candsent.append(ES_result[i])
        
        #candsent's sentence have keywords from question and percent
        
        #Use pos_tag get 'CD' from candsent
        percent_result = []
        for i in range(len(candsent)):
            
            #一sentence by sentence
            postag = nltk.pos_tag(candsent[i])
            
            
            for j in range(len(postag)):
                if 'CD' in postag[j]:
                    percent_result.append(str(postag[j]).replace('[','').replace(']','').replace('\'','').replace(',' , '').replace('(','').replace(')','').replace('CD','').replace('2013','').replace('2014',''))
        
        #return result
        return (max(set(percent_result) , key = percent_result.count)+"percent")
    else :
        return ("Please input the true factor")
    #print (keywords)

In [862]:
#Test this function
find_percent("what percentage change in GDP results from government spending?")

'0.83 percent'

In [863]:
find_percent("what percentage change in GDP results from consumer spending?")

'3.2 percent'

In [865]:
find_percent("what percentage change in GDP results from foreign trade?")

'0.7 percent'

# Define a function to call above function

In [100]:
def answer(question):
    question_type = classifier_question(question)
    
    if question_type == 0:
        return find_bankruptcy(question)
    if question_type == 1:
        return affect_GDP(question)
    if question_type == 2:
        return find_percent(question)
    if question_type == 3:
        return find_CEO(question)

In [924]:
answer("what the  affects GDP?")

'Policy,hurricane,Consumption, consumer spending, government spending, investment, imports, exports, foreign trade,tax'

In [868]:
answer("what the most  affects GDP?")

'policy '

In [872]:
answer("What percentage of drop or increase is associated with this consumer spending?")

'3.2 percent'

In [873]:
answer("What percentage of drop or increase is associated with this government spending?")

'0.83 percent'

In [876]:
answer("What percentage of drop or increase is associated with this foreign trade?")

'0.7 percent'

In [877]:
answer("What is the CEO of Facebook?")

'Mark Zuckerberg'

In [879]:
answer("What is the CEO of Yahoo?")

'Marissa Mayer'

In [880]:
answer("What is the CEO of Amazon?")

'Jeff Bezos'

In [103]:
answer("Which company went bankrupt in September 2008?")

['September', '2008']


'lehman brothers'

In [104]:
answer("Which company went bankrupt in October 2014?")

['October', '2014']


'detroit'

In [105]:
answer("Which company went bankrupt in October 2008?")

['October', '2008']


'seeitmarket'